In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate

        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])

        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)

        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases

        self.x = x
        self.encoded = encoded
        self.decoded = decoded

        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.sub(self.x, self.decoded))))
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
                
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

In [3]:
# from autoencoder import Autoencoder
from sklearn import datasets

hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 3.54863500595
epoch 10: loss = 1.64626097679
epoch 20: loss = 0.769144773483
epoch 30: loss = 0.398946553469
epoch 40: loss = 0.332651883364
epoch 50: loss = 0.319466233253
epoch 60: loss = 0.313293516636
epoch 70: loss = 0.31349208951
epoch 80: loss = 0.303106218576
epoch 90: loss = 0.289905130863
epoch 100: loss = 0.276380091906
epoch 110: loss = 0.261961638927
epoch 120: loss = 0.249286606908
epoch 130: loss = 0.244515880942
epoch 140: loss = 0.24733749032
epoch 150: loss = 0.247637018561
epoch 160: loss = 0.243682786822
epoch 170: loss = 0.238044396043
epoch 180: loss = 0.232438161969
epoch 190: loss = 0.227783888578
epoch 200: loss = 0.224909156561
epoch 210: loss = 0.223893687129
epoch 220: loss = 0.224187806249
epoch 230: loss = 0.225351139903
epoch 240: loss = 0.227103292942
('input', [[8, 4, 6, 2]])
('compressed', array([[ 0.79709238]], dtype=float32))
('reconstructed', array([[ 6.72411633,  2.82488823,  5.98832607,  2.15396857]], dtype=float32))


array([[ 6.72411633,  2.82488823,  5.98832607,  2.15396857]], dtype=float32)